In [1]:
% pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
% load_ext rpy2.ipython

.Rprofile: Setting UK repository


In [3]:
%%R
Temp <- read.table("temporal_behaviors.tsv", sep ="\t",header=TRUE)

In [4]:
%%R
tail(Temp)
dim(Temp)

[1] 46576662       26


In [5]:
%%R
Acct_Purchase <- read.table("acct_purchases_long.tsv", sep ="\t",header=TRUE)

In [6]:
%%R
tail(Acct_Purchase)
dim(Acct_Purchase)

[1] 548118      5


In [7]:
%%R
Acct_Purchase_Test <- read.table("acct_purchases_long_test.tsv", sep ="\t",header=TRUE)

In [8]:
%%R
Acct_Purchase_long <- rbind(Acct_Purchase, Acct_Purchase_Test)
head(Acct_Purchase_long)

  X ACCT_ID                               PROD_NM           CLSD_DATE
1 0      -1                       Analytics Cloud 2015-04-09 00:00:00
2 1      -1                Data.com Premium Clean 2016-01-07 00:00:00
3 2      -1           Data.com Premium Prospector 2016-01-07 00:00:00
4 3      -1                        Desk.com - Pro 2016-04-08 00:00:00
5 4      -1 Employee Community (Force.com Add-On) 2015-04-09 00:00:00
6 5      -1         Force.com - Unlimited Edition 2015-04-09 00:00:00
                                                   PROD_DATE
1                       Analytics Cloud`2015-04-09 00:00:00;
2                Data.com Premium Clean`2016-01-07 00:00:00;
3           Data.com Premium Prospector`2016-01-07 00:00:00;
4                        Desk.com - Pro`2016-04-08 00:00:00;
5 Employee Community (Force.com Add-On)`2015-04-09 00:00:00;
6         Force.com - Unlimited Edition`2015-04-09 00:00:00;


In [9]:
%%R
unique(Acct_Purchase_long$PROD_NM)

 [1] Analytics Cloud                                                        
 [2] Data.com Premium Clean                                                 
 [3] Data.com Premium Prospector                                            
 [4] Desk.com - Pro                                                         
 [5] Employee Community (Force.com Add-On)                                  
 [6] Force.com - Unlimited Edition                                          
 [7] Force.com Platform Embedded Edition                                    
 [8] Premier Success Plan (Support) - Sales Cloud                           
 [9] Premier+ Success Plan (Support & Admin) - Sales Cloud                  
[10] Sales Cloud - Enterprise Edition                                       
[11] Sales Cloud - Professional Edition                                     
[12] Sales Cloud - Unlimited Edition                                        
[13] Sandbox                                                                

In [10]:
%%R
Acct_Purchase_long$DATE <- as.Date(Acct_Purchase_long$CLSD_DATE, format = "%Y-%m-%d")
max(Acct_Purchase_long$DATE)

[1] "2016-04-09"


In [11]:
%%R
# Delete the dataset with invalid account_id
new <- Temp[ ! Temp$ACCOUNT_ID %in% c('0','000000000000000'), ] 
dim(new)

[1] 6920497      26


In [12]:
%%R
# Take the features at '2014-10-01 00:00:00' as base features
base = new[new$QRY_DATE == '2014-05-01 00:00:00',]
dim(base)

[1] 271818     26


In [13]:
%%R
# Take the subset of purchase history of when the 
AC <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Analytics Cloud" ,]
dim(AC)

[1] 641   6


In [14]:
%%R
AC$DATE <- as.Date(AC$CLSD_DATE, format = "%Y-%m-%d")
min(AC$DATE) 

[1] "2013-07-23"


In [15]:
%%R
a <- as.Date("2016-05-1", format = "%Y-%m-%d")

In [16]:
%%R
Temp$DATE <- as.Date(Temp$QRY_DATE, format = "%Y-%m-%d")
min(Temp$DATE)  #'2014-10-01'
Date <- unique(Temp$DATE)   
Date <- Date[order(Date)]
#length(Date)

In [17]:
%%R
Date <- c(Date, a)
Date 

 [1] "2014-05-01" "2014-06-01" "2014-07-01" "2014-08-01" "2014-09-01"
 [6] "2014-10-01" "2014-11-01" "2014-12-01" "2015-01-01" "2015-02-01"
[11] "2015-03-01" "2015-04-01" "2015-05-01" "2015-06-01" "2015-07-01"
[16] "2015-08-01" "2015-09-01" "2015-10-01" "2015-11-01" "2015-12-01"
[21] "2016-01-01" "2016-02-01" "2016-03-01" "2016-03-31" "2016-05-01"


In [18]:
%%R
# Get labels
# Baseline
# Initialze two vectors of lists of length 24
ac_date_list <- vector("list", 25)
label_list <- vector("list", 25)
m <- vector("list", 25)
for (i in 1:25){
    ac_date_list[[i]] <- AC[AC$DATE <= Date[i],] # Get dataframe for accounts which made a purchase before certain dates
    label_list[[i]] <- rep(0,dim(base)[1]) # Initialize the label vector at 0 for each account
    m[[i]] = merge(base, ac_date_list[[i]] , by.x = "ACCOUNT_ID", by.y = "ACCT_ID") 
    label_list[[i]][base$ACCOUNT_ID %in% m[[i]]$ACCOUNT_ID]=1
}

In [19]:
%%R
table(label_list[[25]])


     0      1 
271327    491 


In [20]:
%%R
save(label_list, file = "Label2.RData")

In [22]:
%%R
write.csv( file = "df1.csv",base)

In [37]:
%%R
# Take the subset of purchase history of when the 
pd <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Data.com Premium Clean" ,]#pardot #analytic
dim(pd)

[1] 1293    6


In [21]:
%%R
# Take the subset of purchase history of when the 
pd <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Pardot" ,]#pardot #analytic
dim(pd)

[1] 6202    6


In [22]:
%%R
pd$DATE <- as.Date(pd$CLSD_DATE, format = "%Y-%m-%d")
min(pd$DATE) 

[1] "2013-07-09"


In [23]:
%%R
# Get labels
# Baseline
# Initialze two vectors of lists of length 24
ac_date_list <- vector("list", 25)
label_list_pc <- vector("list", 25)
m <- vector("list", 25)
for (i in 1:25){
    ac_date_list[[i]] <- pd[pd$DATE <= Date[i],] # Get dataframe for accounts which made a purchase before certain dates
    label_list_pc[[i]] <- rep(0,dim(base)[1]) # Initialize the label vector at 0 for each account
    m[[i]] = merge(base, ac_date_list[[i]] , by.x = "ACCOUNT_ID", by.y = "ACCT_ID") 
    label_list_pc[[i]][base$ACCOUNT_ID %in% m[[i]]$ACCOUNT_ID]=1
}

In [24]:
%%R
table(label_list_pc[[25]])


     0      1 
267991   3827 


In [25]:
%%R
save(label_list_pc, file = "Label4.RData")

In [26]:
%%R
# Take the subset of purchase history of when the 
pd <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Sandbox" ,]#pardot #analytic
dim(pd)

[1] 11600     6


In [27]:
%%R
pd$DATE <- as.Date(pd$CLSD_DATE, format = "%Y-%m-%d")
min(pd$DATE) 

[1] "2008-02-20"


In [28]:
%%R
# Get labels
# Baseline
# Initialze two vectors of lists of length 24
ac_date_list <- vector("list", 25)
label_list_pc <- vector("list", 25)
m <- vector("list", 25)
for (i in 1:25){
    ac_date_list[[i]] <- pd[pd$DATE <= Date[i],] # Get dataframe for accounts which made a purchase before certain dates
    label_list_pc[[i]] <- rep(0,dim(base)[1]) # Initialize the label vector at 0 for each account
    m[[i]] = merge(base, ac_date_list[[i]] , by.x = "ACCOUNT_ID", by.y = "ACCT_ID") 
    label_list_pc[[i]][base$ACCOUNT_ID %in% m[[i]]$ACCOUNT_ID]=1
}

In [29]:
%%R
table(label_list_pc[[25]])


     0      1 
263916   7902 


In [30]:
%%R
save(label_list_pc, file = "Label5.RData")

In [35]:
%%R
# Take the subset of purchase history of when the 
pd <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Customer Community" ,]#pardot #analytic
dim(pd)

[1] 4567    6


In [36]:
%%R
pd$DATE <- as.Date(pd$CLSD_DATE, format = "%Y-%m-%d")
min(pd$DATE) 

[1] "2013-06-18"


In [37]:
%%R
# Get labels
# Baseline
# Initialze two vectors of lists of length 24
ac_date_list <- vector("list", 25)
label_list_pc <- vector("list", 25)
m <- vector("list", 25)
for (i in 1:25){
    ac_date_list[[i]] <- pd[pd$DATE <= Date[i],] # Get dataframe for accounts which made a purchase before certain dates
    label_list_pc[[i]] <- rep(0,dim(base)[1]) # Initialize the label vector at 0 for each account
    m[[i]] = merge(base, ac_date_list[[i]] , by.x = "ACCOUNT_ID", by.y = "ACCT_ID") 
    label_list_pc[[i]][base$ACCOUNT_ID %in% m[[i]]$ACCOUNT_ID]=1
}

In [38]:
%%R
table(label_list_pc[[25]])


     0      1 
269024   2794 


In [39]:
%%R
save(label_list_pc, file = "Label6.RData")

In [40]:
%%R
# Take the subset of purchase history of when the 
pd <- Acct_Purchase_long[Acct_Purchase_long$PROD_NM=="Data.com Premium Clean" ,]#pardot #analytic
dim(pd)

[1] 1293    6


In [41]:
%%R
pd$DATE <- as.Date(pd$CLSD_DATE, format = "%Y-%m-%d")
min(pd$DATE) 

[1] "2013-07-07"


In [42]:
%%R
pd$DATE <- as.Date(pd$CLSD_DATE, format = "%Y-%m-%d")
min(pd$DATE) 

[1] "2013-07-07"


In [43]:
%%R
# Get labels
# Baseline
# Initialze two vectors of lists of length 24
ac_date_list <- vector("list", 25)
label_list_pc <- vector("list", 25)
m <- vector("list", 25)
for (i in 1:25){
    ac_date_list[[i]] <- pd[pd$DATE <= Date[i],] # Get dataframe for accounts which made a purchase before certain dates
    label_list_pc[[i]] <- rep(0,dim(base)[1]) # Initialize the label vector at 0 for each account
    m[[i]] = merge(base, ac_date_list[[i]] , by.x = "ACCOUNT_ID", by.y = "ACCT_ID") 
    label_list_pc[[i]][base$ACCOUNT_ID %in% m[[i]]$ACCOUNT_ID]=1
}

In [44]:
%%R
table(label_list_pc[[25]])


     0      1 
270756   1062 


In [45]:
%%R
save(label_list_pc, file = "Label7.RData")

In [2]:
%%R
for (i in 1:10){
    print(i)
}

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


ERROR: Cell magic `%%R` not found.
